In [ ]:
!pip install torch
!pip install transformers
!pip install diffgram
!pip install llama_index
!pip install matplotlib
!pip install tabulate

In [292]:
import json
from diffgram import Project
from llama_index.core import SimpleDirectoryReader, StorageContext
import pandas as pd
from IPython.core.display import display, HTML
import matplotlib as mpl
from tabulate import tabulate
import math
import pprint
import requests
import json

/tmp/ipykernel_10668/3425527988.py:5: DeprecationWarning: Importing display from IPython.core.display is deprecated since IPython 7.14, please import from IPython display
  from IPython.core.display import display, HTML


In [293]:
import torch
from transformers import BertTokenizerFast, BertForTokenClassification
from torch.utils.data import Dataset, DataLoader

In [294]:
host = "https://f91c-2604-3d08-4f7f-e8c0-e41e-1e54-59fe-a985.ngrok-free.app"
project_string_id = "ai_annotate_diffgram_validate"

In [295]:
project_local = Project(host = host,
        project_string_id = project_string_id,
        client_id = "LIVE__va8vrwxb3m13xmme8iwr",
        client_secret = "dm07j8ipzuvumo7dth1iwe0lwhaoywsdkqh7ue94ovrmf58nnct730an49ij")

## Utility functions to create/check a directory or file in diffgram with dataset_name

In [296]:
def check_if_directory_exist(dir_name):
    project = project_local.directory.get_directory_list(limit=50000)
    for project_dir in project:
        if (project_dir.__dict__['nickname'] == dir_name):
            return project_dir
    return None

In [297]:
## You may need to run this twice to see if the directory is created
def create_diffgram_directory(dataset_name):
    #directory = project_local.directory.get(name = dataset_name)
    directory = check_if_directory_exist(dataset_name)
    if (directory is None):
        project_local.directory.new(name=dataset_name)
        directory = check_if_directory_exist(dataset_name)
        print(directory.__dict__)    
    return directory

In [298]:
## check if file exist in the dir
def check_if_file_exist_in_dir(filename):
    file = project_local.file.file_list_exists(filename)

In [299]:
## check if file exist in the dir
def check_if_file_exist_in_dir(filename):
    file = project_local.file.file_list_exists(filename)

In [300]:
def create_dataset_job(data_suffix, job_suffix, index, member_list_ids):
    dataset_batch_name = data_suffix + str(index)
    directory = create_diffgram_directory(dataset_batch_name)

    if directory is None:
        print(f"{dataset_batch_name} Directory does not exist")
        return

    job_name = job_suffix + str(index)

    job = project_local.job.new(
        name = job_name,
        instance_type = "box",
        share = "Project",
        sync_directories = [directory],
        label_schema_id = schema_id,
        tag_list = ["Laws", "Acts", "Regulations"],
        members_list_ids = member_list_ids,
        auto_launch = True
    )
    print(f"The {job_name} task is created")
    return directory

In [301]:
def upload_files_to_dataset(index,batch_size, offset, directory):
    for document in range((index+offset) * batch_size, ((index + offset) * batch_size) + batch_size):
        filename = diffgram_documents[document].metadata['filename']
        # check if the file exist in the diffgram directory
        try:
            file = project_local.file.from_local(filename,directory_id=directory.__dict__['id'])
        except:
            print(f"File with {filename} exist in this directory. Continuing ....")
            continue;

## Import all the files 
### make sure you have the diffgram_processing_v2 folder which has all the data arranged for NER task

In [302]:
from llama_index.core import SimpleDirectoryReader, StorageContext

In [303]:
file_metadata = lambda x: {"filename": x}
diffgram_documents = SimpleDirectoryReader("diffgram_processing_v2",file_metadata=file_metadata).load_data()

In [304]:
print(len(diffgram_documents))

81612


In [305]:
print(diffgram_documents[500].get_text())

Chunk ID: Great Bear Rainforest (Forest Management) Act-chunk-Application of Forest Act to partition orders-0000
Act ID: Great Bear Rainforest (Forest Management) Act
Regulation ID: None
Section Name: Application of Forest Act to partition orders
Section ID: 36
Sequence ID: 0
Text:
1 a partition order that relates to an aac partition specified by the lieutenant governor in council for a relevant timber supply area is deemed to be an order made by the minister under section 75. 02 ( 2 ) [ partition order ] of the forest act. 2 a partition order that relates to an aac partition specified by the lieutenant governor in council for a relevant tree farm licence area is deemed to be an order made by the minister under section 75. 02 ( 3 ) of the forest act.



## Fetch and add the schema

In [306]:
f = open('legal_ner_schema.json')
NER_schema = json.load(f)
print(NER_schema)

{'schema': [{'name': 'O', 'description': 'Outside of named entity (default)', 'example': 'The'}, {'name': 'B-INTERNAL_REF_SECTION', 'description': 'Beginning of internal section reference', 'example': "Section in 'Section 101'"}, {'name': 'I-INTERNAL_REF_SECTION', 'description': 'Inside of internal section reference', 'example': "101 in 'Section 101'"}, {'name': 'B-EXTERNAL_REF_ACT_SECTION', 'description': 'Beginning of external act section reference', 'example': "Section in 'Section 5 of the Clean Air Act'"}, {'name': 'I-EXTERNAL_REF_ACT_SECTION', 'description': 'Inside of external act section reference', 'example': "5 in 'Section 5 of the Clean Air Act'"}, {'name': 'B-EXTERNAL_REF_REGULATION_SECTION', 'description': 'Beginning of external regulation section reference', 'example': "Section in 'Section 3.2 of the Federal Acquisition Regulation'"}, {'name': 'I-EXTERNAL_REF_REGULATION_SECTION', 'description': 'Inside of external regulation section reference', 'example': "3.2 in 'Section 

In [307]:
## Id the Labels
# Create label to ID mapping
label2id = []
id2label = []
for id, label in enumerate(NER_schema['schema']):
    label2id.append(id)
    id2label.append(NER_schema['schema'][id]['name'])
print(label2id)
print(id2label)

[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22]
['O', 'B-INTERNAL_REF_SECTION', 'I-INTERNAL_REF_SECTION', 'B-EXTERNAL_REF_ACT_SECTION', 'I-EXTERNAL_REF_ACT_SECTION', 'B-EXTERNAL_REF_REGULATION_SECTION', 'I-EXTERNAL_REF_REGULATION_SECTION', 'B-REF_TYPE', 'I-REF_TYPE', 'B-EXCLUDED_SECTION', 'I-EXCLUDED_SECTION', 'B-INCLUDED_SECTION', 'I-INCLUDED_SECTION', 'B-DECISION', 'I-DECISION', 'B-ACT_NAME', 'I-ACT_NAME', 'B-REGULATION_NAME', 'I-REGULATION_NAME', 'B-SUBSECTION', 'I-SUBSECTION', 'B-PARAGRAPH', 'I-PARAGRAPH']


In [308]:
NER_schema_name = 'NER_TRAINING_SCHEMA'
schema_id = None
schemas = project_local.schema.list()
print(json.dumps(schemas, indent=2))
for schema in schemas:
    if schema['name'] == NER_schema_name:
        schema_id = schema['id']
if schema_id == None:
    json_response = project_local.new_schema(name=NER_schema_name)
    schema_id = json_response["id"]
    for NER in NER_schema['schema']:
        print(NER['name'])
        project_local.label_new(NER, schema_id=schema_id)

[
  {
    "archived": false,
    "id": 6,
    "is_default": true,
    "member_created_id": 1,
    "member_updated_id": null,
    "name": "Default Schema",
    "project_id": 3,
    "time_created": "2024-10-10 16:33:25",
    "time_updated": null
  },
  {
    "archived": false,
    "id": 7,
    "is_default": false,
    "member_created_id": 5,
    "member_updated_id": null,
    "name": "NER_TRAINING_SCHEMA",
    "project_id": 3,
    "time_created": "2024-10-10 16:45:08",
    "time_updated": null
  }
]


## Get the NER Labels and ID's as this would be needed to pre-annotate the data

In [309]:
schema_list = project_local.schema.list()

In [310]:
# Retrieve the list of schemas
schemas = project_local.schema.list()
schema_NER_id = None
# Loop through each schema and print schema value names and their IDs
for schema in schemas:
    if schema['name'] == NER_schema_name:
        print(f"Schema Name: {schema['name']}, Schema ID: {schema['id']}")
        schema_NER_id = schema['id']
        # If schema contains schema values, print their names and IDs
        if 'schema_values' in schema and schema['schema_values']:
            for schema_value in schema['schema_values']:
                print(f"\tSchema Value Name: {schema_value['name']}, Schema Value ID: {schema_value['id']}")


Schema Name: NER_TRAINING_SCHEMA, Schema ID: 7


In [311]:
def get_schema_list(id):
    auth = project_local.session.auth
    url = f"{host}/api/project/{project_string_id}/labels?schema_id={id}"
    # Step 4: Make the POST request using the SDK's session auth
    response = requests.get(url, auth=auth)
    # Step 5: Handle the response
    if response.status_code == 200:
        #print("Annotation update successful!")
        #pprint.pprint(response.json())  # View the updated data
        return response.json()
    else:
        print(f"Error: {response.status_code}")
        print(response.text)  # Print error details for debugging

In [312]:
print(schema_NER_id)

7


In [313]:
schema_labels = get_schema_list(schema_NER_id)

In [314]:
schema_label_id_value = []

In [315]:
if schema_labels is not None:
    labels = schema_labels['labels_out']
    for label in labels:
        value = {}
        value['id'] = label['id']
        value['name'] = label['label']['name']
        schema_label_id_value.append(value)

In [316]:
pprint.pprint(schema_label_id_value)

[{'id': 10907, 'name': 'O'},
 {'id': 10908, 'name': 'B-INTERNAL_REF_SECTION'},
 {'id': 10909, 'name': 'I-INTERNAL_REF_SECTION'},
 {'id': 10910, 'name': 'B-EXTERNAL_REF_ACT_SECTION'},
 {'id': 10911, 'name': 'I-EXTERNAL_REF_ACT_SECTION'},
 {'id': 10912, 'name': 'B-EXTERNAL_REF_REGULATION_SECTION'},
 {'id': 10913, 'name': 'I-EXTERNAL_REF_REGULATION_SECTION'},
 {'id': 10914, 'name': 'B-REF_TYPE'},
 {'id': 10915, 'name': 'I-REF_TYPE'},
 {'id': 10916, 'name': 'B-EXCLUDED_SECTION'},
 {'id': 10917, 'name': 'I-EXCLUDED_SECTION'},
 {'id': 10918, 'name': 'B-INCLUDED_SECTION'},
 {'id': 10919, 'name': 'I-INCLUDED_SECTION'},
 {'id': 10920, 'name': 'B-DECISION'},
 {'id': 10921, 'name': 'I-DECISION'},
 {'id': 10922, 'name': 'B-ACT_NAME'},
 {'id': 10923, 'name': 'I-ACT_NAME'},
 {'id': 10924, 'name': 'B-REGULATION_NAME'},
 {'id': 10925, 'name': 'I-REGULATION_NAME'},
 {'id': 10926, 'name': 'B-SUBSECTION'},
 {'id': 10927, 'name': 'I-SUBSECTION'},
 {'id': 10928, 'name': 'B-PARAGRAPH'},
 {'id': 10929, 'name

## Fetch the files and add it to diffgram
if you want to insert one file at a time
    file = project_local.file.from_local("diffgram_processing_v2/800.txt")

### Separating the data into train and test
1. Batch each train or test data to a size of 64 or any multiple of 2^ (easier for memory alignment/ faster processing)
2. To make the netowkr good at NER the more data it is trained on the better. IF we want to train it with 5000 examples, we need to create 5000/64 batches of test data
3. We can then have 5% * 5000 as the test data
4. we can name the diffgram dataset as NER_train_batch_x where x is the batch number
5. similar we can name the diffgram dataset for test as NER_test_batch_x 
6. After creating the dataset we create the task for both test and train e.g. NER_train_JOB_. 
7. The task links the dataset and the schema.
8. After which we add files to the dataset based on the batch size
9. Repeat steps 4 - 8 till all the data is populated and arranged for annotation

In [50]:
MAX_NUM_OF_TASK = 3 #To keep the dataset small for now

In [51]:
def create_datasets(diffgram_documents, num_training_data, batch_size, train_suffix, test_suffix, job_train_suffix, job_test_suffix):
    #check if the lenght of all the data is equal to or more than num_training_data
    if (len(diffgram_documents) < num_training_data):
        print(f"Not sufficient data for training {len(diffgram_documents)}")
        return
    
    train_batch_size = math.floor(num_training_data/batch_size)
    test_batch_size = math.floor((num_training_data * (5 /100))/ batch_size)
    #train_dataset_name = "NER_train_batch_"
    
    print(f"The batch size of the training data is : {train_batch_size}")
    print(f"The batch size of the test data is: {test_batch_size}")
    
    member_list = project_local.get_member_list()
    member_list_ids = [x['member_id'] for x in member_list]
    
    #schemas = project_local.schema.list()
    train_batch_size = max(MAX_NUM_OF_TASK, train_batch_size)
    test_batch_size = max(MAX_NUM_OF_TASK, test_batch_size)
    
    for index in range(0, train_batch_size):
        directory = create_dataset_job(train_suffix, job_train_suffix, index, member_list_ids)
        
        print(f"Creating / Uploading data to directory {directory.__dict__['nickname']}")
        upload_files_to_dataset(index,batch_size, 0, directory)
        
    for index in range(0, test_batch_size):
        directory = create_dataset_job(test_suffix, job_test_suffix, index, member_list_ids)
        
        print(f"Creating / Uploading data to directory {directory.__dict__['nickname']}")
        upload_files_to_dataset(index,batch_size, train_batch_size+1, directory)

In [317]:
NUM_TRAINING_DATA = 5000
BATCH_SIZE = 32
train_dataset_suffix = "NER_train_batch_"
test_dataset_suffix = "NER_test_batch_"
JOB_NAME = "Law_NER_task1"
JOB_TRAIN_SUFFIX = "NER_train_JOB_"
JOB_TEST_SUFFIX = "NER_test_JOB_"

In [ ]:
create_datasets(diffgram_documents, NUM_TRAINING_DATA, BATCH_SIZE, train_dataset_suffix, test_dataset_suffix,  JOB_TRAIN_SUFFIX, JOB_TEST_SUFFIX)

## Import annotated data

- Get a file and send it to openai to annotate
- After the file is annotated save it back to diffgram to view the annotation

In [318]:
def get_job_names(diffgram_documents, num_training_data, batch_size, JOB_SUFFIX):
    if (len(diffgram_documents) < num_training_data):
        print(f"Not sufficient data for training {len(diffgram_documents)}")
        return
    
    num_batches = math.floor(num_training_data/batch_size)
    jobs = []
    for index in range(0, num_batches):
        jobs_name = JOB_SUFFIX + str(index)
        jobs.append(jobs_name)
    return jobs

In [319]:
jobs = get_job_names(diffgram_documents, NUM_TRAINING_DATA, BATCH_SIZE, JOB_TRAIN_SUFFIX)

In [320]:
get_job = project_local.job.list(limit=10000, page_number=1)

In [321]:
jobs_with_data_index = []
for job_key, job_list in enumerate(get_job):
    try:
        nickname = job_list['attached_directories_dict']['attached_directories_list'][0]['nickname']
        if nickname:
            jobs_with_data_index.append(job_key)
        #print(nickname)
    except KeyError:
        print("Key not found.")
    except IndexError:
        print("List index out of range.")

List index out of range.


In [322]:
results = project_local.job

In [323]:
print(jobs_with_data_index)

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19]


## Pre-annotating for each job
 - For  each job we need to extract the files
 - For each file we need to formulate the data to send to 3rd party
 - once the data is annotated we need send it back to diffgram

In [324]:
def get_file_number(completed_annotations, files_index_in_job):
    data = []
    sentences = []
    labels = []
    data_index = 0
    for completed_annotation in completed_annotations:
        try:
            file_index = int(completed_annotation)
            files_index_in_job.append(completed_annotation)
            #print(completed_annotation)
            continue
        except:
            #print(f"{completed_annotation} is not a file")
            continue
            
        #print(f"{completed_annotation} ----")
        if (completed_annotation != 'attribute_groups_reference')  \
            and (completed_annotation != 'export_info') \
            and (completed_annotation != 'label_map') \
            and (completed_annotation != 'readme') \
            and (completed_annotation != 'label_colour_map'):
            sentence_local = []
            labels_local = []

            # First get the point where the annotation is started
            for start in completed_annotations[completed_annotation]['instance_list']:
                if 'start_token' in start:
                    start_token =  start['start_token']
                    break

            #start_token = completed_annotations[completed_annotation]['instance_list'][0]['start_token']
            for annotated_index in range(start_token, len(completed_annotations[completed_annotation]['text']['tokens']['words'])):
                # check if this text is annotated
                for data in completed_annotations[completed_annotation]['instance_list']:
                    if 'start_token' in data:
                        if annotated_index == data['start_token']:
                            sentence_local.append(completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value'])
                            labels_local.append(completed_annotations['label_map'][str(data['label_file_id'])])
                            #print(f"{completed_annotations[completed_annotation]['text']['tokens']['words'][annotated_index]['value']} - {completed_annotations['label_map'][str(data['label_file_id'])]}")
                            break;
            sentences.append(sentence_local)       
            labels.append(labels_local)
            data_index+=1

In [325]:
def extract_word_data(url):
    # Original URL with localhost
    # Replace localhost with ngrok URL (example: "https://example.ngrok.io")
    file_url = url.replace("http://localhost:8085", host)

    # Make the GET request to fetch the file
    response = requests.get(file_url)

    # Check if the request was successful
    if response.status_code == 200:
        # Parse the JSON content into a Python dictionary
        data = response.json()  # Assuming the file is in JSON format
        return data
    else:
        print(f"Failed to retrieve the file. Status code: {response.status_code}")

In [326]:
def print_data(data):
    #print(data['nltk']['words'])
    for start_index, token in enumerate(data['nltk']['words']):
        print(f"Index: {start_index} Word: {token['value']}") 

In [327]:
def send_preannotation_to_diffgram(file):
    # Step 1: Extract the session's auth credentials (client_id and client_secret)
    auth = project_local.session.auth

    # Step 2: Define the API URL for the custom annotation update endpoint
    file_id = file.id  # Replace with your file ID
    #project_string_id = "your_project_string_id"  # Replace with your project string ID
    url = f"{host}/api/project/{project_string_id}/file/{file_id}/annotation/update"

    # Step 3: Define the data (e.g., instance_list) for updating annotations
    data = {
        "instance_list": file.__dict__['instance_list']
    }

    # Step 4: Make the POST request using the SDK's session auth
    response = requests.post(url, json=data, auth=auth)

    # Step 5: Handle the response
    if response.status_code == 200:
        print("Annotation update successful!")
        print(response.json())  # View the updated data
    else:
        print(f"Error: {response.status_code}")
        print(response.text)  # Print error details for debugging

In [331]:
def get_label_value(search_term):
    for obj in schema_label_id_value:
        if search_term == obj['name']:
            print(f"Found Match - ID: {obj['id']}, Name: {obj['name']}")
            return obj['id']
    return get_label_value("O")   

In [336]:
get_label_value('B-INTERNAL_REF_SECTION')

Found Match - ID: 10908, Name: B-INTERNAL_REF_SECTION


10908

In [337]:
def create_preannotation(file):
    """ This function should call a supeiror A.I for annotation
    """
    #instance_list = []
    print("Creating annotation")
    demo_instance_list ={
        #Z"id": 10907,  # Increment id starting from 20160
        "label_file_id":get_label_value("O"),
        "start_token": 0,
        "end_token": 0,
        "type": 'text_token', # Optional: Add the token text
        #"id":23,
         #"directory_id":165
    }
    file.__dict__['instance_list'].append(demo_instance_list)
    #instance_list.append(demo_instance_list)
    #return file

In [338]:
def annotate_data(file, data):
    if (file.__dict__['instance_list']):
        for key, instance_list in enumerate():
            pprint.pprint(f" { key} {instance_list['start_token']}   {instance_list['id']} {instance_list['label_file_id']}")
            # Create annotation
            create_preannotation(file)
            print(file.__dict__['instance_list'])
            send_preannotation_to_diffgram(file)
    else:
        print("instance list not found")
        file.__dict__['instance_list'] = []
        create_preannotation(file)
        print(file.__dict__['instance_list'])
        send_preannotation_to_diffgram(file)

In [339]:
def process_each_file_for_pre_annotation(files_index_in_job):
    for file_index in files_index_in_job:
        file = project_local.file.get_by_id(file_index,with_instances=True)
        url = file.__dict__['text']['tokens_url_signed']
        #print(url)
        data = extract_word_data(url)
        #print_data(data)
        annotate_data(file, data)

In [340]:
for job_index in jobs_with_data_index:
    print(job_index)
    results.refresh_from_dict(get_job[job_index])
    try:
        completed_annotations = results.generate_export()
        files_index_in_job = []
        get_file_number(completed_annotations, files_index_in_job)
        process_each_file_for_pre_annotation(files_index_in_job)
        #print(files_index_in_job)
        #print(len(completed_annotations))
        print("\n")    
        break
    except:
        print("Not able to export data")

1
Not able to export data
2
Not able to export data
3
instance list not found
Creating annotation
Found Match - ID: 10907, Name: O
[{'label_file_id': 10907, 'start_token': 0, 'end_token': 0, 'type': 'text_token'}]
Annotation update successful!
{'added_instances': [{'action_type': 'created', 'angle': 0.0, 'attribute_groups': None, 'bounds': None, 'bounds_lonlat': None, 'center_3d': {}, 'center_x': None, 'center_y': None, 'change_source': None, 'coords': None, 'cp': None, 'created_time': '2024-10-17T16:43:39.543796', 'creation_ref_id': None, 'deleted_time': None, 'deletion_type': None, 'dimensions_3d': {}, 'edges': [], 'end_time': None, 'end_token': 0, 'file_id': 11058, 'from_instance_id': None, 'front_face': None, 'height': None, 'id': 20651, 'interpolated': False, 'label_file_id': 10907, 'lonlat': None, 'machine_made': False, 'member_created_id': 8, 'model_id': None, 'model_run_id': None, 'next_id': None, 'nodes': [], 'number': None, 'p1': None, 'p2': None, 'pause_object': None, 'point